In [ ]:
using Plots
using TensorCrossInterpolation, LinearAlgebra
using QuanticsTCI
using QuanticsGrids
using ITensorMPS
using ITensors
using NDTensors
using TCIITensorConversion
using IterativeSolvers
using LinearMaps
using LinearAlgebra
include("src/differential_mpo.jl")

In [ ]:
# simulation parameter
R = 10
max_bond = 33
tol = 1e-5
cutoff = 1e-12
dx = 1 / (2^R-1)
delta_t = 0.1 * 2.0^-(R-1)
nu = 1e-5
penalty_coefficient = 2.5e5

# define grid
grid = DiscretizedGrid{2}(R, (0,0), (1,1); includeendpoint = true)

# define sites
s = siteinds("Qudit", R, dim=4)

In [ ]:
# initial conditions

x_min = 0.4
x_max = 0.6
h = 1/200

function d_1_func(x, y)
    out = 2 / h^2 * ( (y - x_max) * exp( -(y - x_max)^2 / h^2 ) + (y - x_min) * exp( -(y - x_min)^2 / h^2 ) )
    return out * ( sin(8π * x) + sin(24π * x) + sin(6π * x) ) 
end

function d_2_func(x, y)
    out = π * ( exp( -(y - x_max)^2 / h^2 ) + exp( -(y - x_min)^2 / h^2 ) )
    return out * ( 8*cos(8π * x) + 24*cos(24π * x) + 6*cos(6π * x) ) 
end

function A_func(x, y)
    return sqrt( d_1_func(x,y)^2 + d_2_func(x,y)^2 )
end

A = find_max_on_2D_grid(A_func, R)
u_0 = 1.
delta_var = u_0 / (40 * A)

function J_1_func(y)
    return u_0 / 2 * ( tanh( (y - x_min)/h ) - tanh( (y - x_max)/h ) - 1 )
end

function D_1_func(x, y)
    return delta_var * d_1_func(x, y)
end

function D_2_func(x, y)
    return delta_var * d_2_func(x, y)
end

function u_init_1_func(x, y)
    return J_1_func(y) + D_1_func(x, y)
end

function u_init_2_func(x, y)
    return D_2_func(x, y)
end

In [ ]:
 # build mps with QuanticsTCI
u1q, ranks1, errors1 = quanticscrossinterpolate(Float64, u_init_1_func, grid; maxbonddim=max_bond)
u2q, ranks2, errors2 = quanticscrossinterpolate(Float64, u_init_2_func, grid; maxbonddim=max_bond)

# convert to ITensorMPS format
u1 = deepcopy(ITensorMPS.MPS(TensorTrain(u1q.tci), sites=s))
u2 = deepcopy(ITensorMPS.MPS(TensorTrain(u2q.tci), sites=s))
v1 = deepcopy(u1)
v2 = deepcopy(u2)

# build base MPO's
d1x = Diff_1_8_x(dx, s)
d1y = Diff_1_8_y(dx, s)
d2x = Diff_2_8_x(dx, s)
d2y = Diff_2_8_y(dx, s)
dx_dx = apply(d1x, d1x, maxdim=max_bond, cutoff=cutoff)
dy_dy = apply(d1y, d1y, maxdim=max_bond, cutoff=cutoff)
dx_dy = apply(d1x, d1y, maxdim=max_bond, cutoff=cutoff)
del = MPO([delta(s[i], s[i]', s[i]'') for i in 1:length(s)])

plot_mps(u2, R, 6)

In [ ]:
center = 1

# orthogonalize mps's
v1 = orthogonalize(v1, center)
v2 = orthogonalize(v2, center)
a1 = orthogonalize(u1, center)
a2 = orthogonalize(u2, center)
b1 = orthogonalize(u1, center)
b2 = orthogonalize(u2, center)

# starting guess for cg
c_vec = get_c_vec(v1, v2, center)

# rhs of equation
beta = make_beta(v1, v2, a1, a2, b1, b2, center, delta_t, nu, d1x, d1y, d2x, d2y, del, max_bond, cutoff)

# linear operator
function A_function(c_vec)
    H_c = apply_H(c_vec, v1, v2, dx_dx, dy_dy, dx_dy, center, max_bond, cutoff)
    return c_vec - penalty_coefficient * delta_t^2 * H_c
end
d = prod(size(v1[center])) + prod(size(v2[center]))
A = FunctionMap{Float64,false}(A_function, d)

In [ ]:
# c_vec = get_c_vec(v1, v2, center) # restart iteration
error = norm(A(c_vec) - beta) / sqrt(length(c_vec))
history = cg!(c_vec, A, beta, abstol=0, reltol=1e-6, verbose=false, maxiter=100, log=true)[2]
println(history)
new_error = norm(A(c_vec) - beta) / sqrt(length(c_vec))
println("Old error: $(error)\nNew error: $(new_error)\nRelative improvement: $(new_error/error)")

In [ ]:
v1, v2 = insert_c_vec(c_vec, v1, v2, center)
println("Done!")

In [ ]:
# solve system
info = true
max_iter = 10
error = norm(A(c_vec) - beta) / length(c_vec)

for i in 1:max_iter
    cg!(c_vec, A, beta, abstol=0, reltol=tol, verbose=false, maxiter=100, log=false)
    error_new = norm(A(c_vec) - beta) / length(c_vec)

    if info
        println("Step $(i), new error = $(error_new)")
    end

    if error_new > error
        println("Failed at step $(i), error: $(error_new)")
        break
    else
        error = error_new
    end

    if error < tol
        println("Converged at step $(i), error: $(error_new)")
        break
    end
end

In [ ]:
v1, v2 = insert_c_vec(c_vec, v1, v2, center)

In [ ]:
A_mat = Matrix(A)
display(eigvals(A_mat))

In [ ]:
diff = abs.(A_mat - transpose(A_mat))
println("Max difference: $(maximum(diff))")
println("Mean diff: $(sum(diff) / length(diff))")
display(A_mat)

In [ ]:
function optimize_V(v, a, b, del_t, log=false)
    success = true
    for center in length(v[1]):-1:1
        
        
        v = orthogonalize.(v, center)
        a = orthogonalize.(a, center)
        b = orthogonalize.(b, center)

        # starting guess for cg
        c_vec = get_c_vec(v, center)

        # rhs of equation
        beta = make_beta(v, a, b, center, delta_t, nu, d1, d2, del, max_bond)

        # linear operator
        function A_function(c_vec)
            return c_vec - penalty_coefficient * delta_t^2 * apply_H(c_vec, v, d1, center, max_bond)
        end
        d = sum(length.(array.(getindex.(v, center))))
        A = FunctionMap{Float64,false}(A_function, d)

        # solve system
        if log
            history = cg!(c_vec, A, beta, abstol=1e-6, reltol=0, verbose=false, maxiter=100, log=log)[2]
            println("Center $center: $history")
        else
            cg!(c_vec, A, beta, abstol=1e-3, reltol=0, verbose=false, maxiter=100)
        end

        if norm(A(c_vec) - beta) > 1e-2
            println("Fail!")
            success = false
            break
        end

        # place solution back in the mps
        place_c_vec!(v, c_vec, center)
    end
    return v, success
end

In [ ]:
U1, success1 = optimize_V(u, u/4, u, delta_t/6, true)
U2, success2 = optimize_V(u, u/4, 3*U1 + u/4, delta_t/3, true)
U3, success3 = optimize_V(u, u/4, 3/2*U2 + 5/8*u, delta_t/3, true)
U4, success4 = optimize_V(u, u/4, 3*U3 + u/4, delta_t/6, true)

if success1 && success2 && success3 && success4
    println("Timestep done!")
    u = U1 + U2 + U3 + U4
else
    println("Fail! End")
end

In [ ]:
plot_mps(u[1], R, 6)